In [152]:
import sqlite3
import pandas as pd
import numpy as np
from lxml import etree
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString

### [data set][kaggle]

#### 1 - I want to do a model to predict simple outcomes from a soccer match, such as number of corners, or number of shots on goal, it can be based on both teams, or just home team, or away team, or combined teams.  
#### 2 - This can be used by bet industry, even further to boards or associations that want to track and audit matches (as outcomes may be indications of forged matches), also can be used within the scout and analyst teams in the clubs  
#### 3 - Think logistic classifier but I am open to directions
#### 4 - Tabular data
#### 5 - This data set contains 8 seasons, 380 matches each, so a total of 3,040 rows, if necessary to limit number of rows, we need just to exclude some seasons. Then we have 18 columns, but two of them won't be used because are match_id and season, then we have two string columns, names of home team and away team, and all the other 14 columns are integers. If we combine those 14 columns to use just straight outcomes for goals or fouls or shots on goal, we have indeed 7 columns.
#### 6 - I already used sqlite3, pandas and lxml to extract, clean and load the data wanted. Probably will use scikit-learn, matplotlib and seaborn to make some visuals  
#### 7 - Hope two get results from two or more algorithms.

[kaggle]: https://www.kaggle.com/datasets/hugomathien/soccer/data

In [153]:
data = sqlite3.connect('database.sqlite')

In [154]:
leagues = pd.read_sql("""
                      SELECT id, name
                      FROM League
                      ORDER BY id
                      """, data)
leagues

,id,name
0,1,Belgium Jupiler League
1,1729,England Premier League
2,4769,France Ligue 1
3,7809,Germany 1. Bundesliga
4,10257,Italy Serie A
5,13274,Netherlands Eredivisie
6,15722,Poland Ekstraklasa
7,17642,Portugal Liga ZON Sagres
8,19694,Scotland Premier League
9,21518,Spain LIGA BBVA


In [155]:
matches = pd.read_sql_query(
                            """
                            SELECT Match.match_api_id AS match_id
                                 , Match.season
                                 , HomeTeam.team_long_name AS home_team
                                 , Match.home_team_goal
                                 , AwayTeam.team_long_name AS away_team
                                 , Match.away_team_goal 
                                 , Match.home_team_api_id, Match.away_team_api_id
                                 , Country.name AS country_name
                                 , Match.goal, Match.shoton
                                 , Match.shotoff, Match.foulcommit, Match.card 
                                 , Match.cross, Match.corner, Match.possession
                            FROM Match
                                 JOIN Country ON Match.country_id = Country.id
                                 JOIN Team AS HomeTeam ON Match.home_team_api_id = HomeTeam.team_api_id
                                 JOIN Team AS AwayTeam ON Match.away_team_api_id = AwayTeam.team_api_id
                            WHERE Match.league_id=1729;
                            """, data
                           )

 

In [156]:
matches

,match_id,season,home_team,home_team_goal,away_team,away_team_goal,home_team_api_id,away_team_api_id,country_name,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
0,489042,2008/2009,Manchester United,1,Newcastle United,1,10260,10261,England,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>56</comment><event...
1,489043,2008/2009,Arsenal,1,West Bromwich Albion,0,9825,8659,England,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card />,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>65</comment><event...
2,489044,2008/2009,Sunderland,0,Liverpool,1,8472,8650,England,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>45</comment><event...
3,489045,2008/2009,West Ham United,2,Wigan Athletic,1,8654,8528,England,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>50</comment><event...
4,489046,2008/2009,Aston Villa,4,Manchester City,2,10252,8456,England,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><corners>1</corners></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>51</comment><event...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,1988795,2015/2016,Southampton,2,Leicester City,2,8466,8197,England,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>54</comment><stats...
3036,1988796,2015/2016,Swansea City,0,Stoke City,1,10003,10194,England,<goal><value><comment>p</comment><stats><penal...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>53</comment><stats...
3037,1988797,2015/2016,Tottenham Hotspur,0,Liverpool,0,8586,8650,England,<goal />,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>45</comment><stats...
3038,1988798,2015/2016,Watford,0,Arsenal,3,9817,9825,England,<goal><

### Next 4 codes are from this [page][website]

[website]: https://www.kaggle.com/code/njitram/exploring-the-incident-data

In [157]:
def value_from_xpath(element, xpath, to_int=False, index=1):
    try:
        result = element.xpath(xpath)[index-1].text
        if to_int:
            return int(result)
        else:
            return result
    except IndexError:
        return np.nan

In [158]:
def node_to_dataframe(n, key):
    return pd.DataFrame({
        "id": value_from_xpath(n, "./id", to_int=True),
        #"type": value_from_xpath(n, "./type"),
        "subtype1": value_from_xpath(n, "./subtype"),
        #"subtype2": value_from_xpath(n, "./" + key + "_type"),
        #"player1": value_from_xpath(n, "./player1"),
        #"player2": value_from_xpath(n, "./player2"),
        "awaypos": value_from_xpath(n, "./awaypos"),
        "homepos": value_from_xpath(n, "./homepos"),
        "team": value_from_xpath(n, "./team")
        }, index=[0])

In [159]:
possession_incidents = []

# Loop over all matches and maping the incidents
for key in ["possession"]:
    for _, match in matches.iterrows():
        if pd.notnull(match[key]):
            xml = etree.fromstring(match[key])
            for n in xml.xpath("/" + key + "/value"):
                df = node_to_dataframe(n, key)
                df["match_id"] = match["match_id"]
                possession_incidents.append(df)

In [160]:
possession = pd.concat(possession_incidents, ignore_index=True)

In [161]:
possession

,id,subtype1,awaypos,homepos,team,match_id
0,379029,possession,44,56,NaN,489042
1,379251,possession,46,54,NaN,489042
2,379443,possession,46,54,NaN,489042
3,379575,possession,45,55,NaN,489042
4,375608,possession,35,65,NaN,489043
...,...,...,...,...,...,...
13577,4914159,possession,67,33,NaN,1988798
13578,4909229,possession,50,50,NaN,1988799
13579,4909837,possession,47,53,NaN,1988799
13580,4910603,possession,44,56,NaN,1988799


In [162]:
possession = possession.groupby('match_id').last().reset_index()

In [163]:
possession

,match_id,id,subtype1,awaypos,homepos,team
0,489042,379575,possession,45,55,NaN
1,489043,375909,possession,34,66,NaN
2,489044,378069,possession,54,46,NaN
3,489045,377621,possession,48,52,NaN
4,489046,379254,possession,48,52,NaN
...,...,...,...,...,...,...
3030,1989075,5591957,possession,39,61,NaN
3031,1989076,5577250,possession,67,33,NaN
3032,1989077,5580867,possession,69,31,NaN
3033,1989078,5588445,possession,30,70,NaN


In [164]:
# Function that converts nodes to dataframe
def node_to_dataframe(n, key):
    return pd.DataFrame({
        "id": value_from_xpath(n, "./id", to_int=True),
        "type": value_from_xpath(n, "./type"),
        "subtype1": value_from_xpath(n, "./subtype"),
        "subtype2": value_from_xpath(n, "./" + key + "_type"),
        "player1": value_from_xpath(n, "./player1"),
        "player2": value_from_xpath(n, "./player2"),
        "team": value_from_xpath(n, "./team"),
        }, index=[0])

# "lon": value_from_xpath(n, "./coordinates/value", to_int=True, index=1),
# "lat": value_from_xpath(n, "./coordinates/value", to_int=True, index=2),
# "elapsed": value_from_xpath(n, "./elapsed", to_int=True),
# "elapsed_plus": value_from_xpath(n, "./elapesed_plus", to_int=True)

In [165]:
incidents = []

# Loop over all matches and maping the incidents
for key in ["goal", "card", "foulcommit", "shoton", "shotoff", "cross", "corner"]:
    for _, match in matches.iterrows():
        if pd.notnull(match[key]):
            xml = etree.fromstring(match[key])
            for n in xml.xpath("/" + key + "/value"):
                df = node_to_dataframe(n, key)
                df["match_id"] = match["match_id"]
                incidents.append(df)

In [166]:
# Run if you have time
actions = pd.concat(incidents, ignore_index=True)

In [167]:
actions

,id,type,subtype1,subtype2,player1,player2,team,match_id
0,378998,goal,header,n,37799,38807,10261,489042
1,379019,goal,shot,n,24148,24154,10260,489042
2,375546,goal,shot,n,26181,39297,9825,489043
3,378041,goal,distance,n,30853,30889,8650,489044
4,376060,goal,shot,n,23139,36394,8654,489045
...,...,...,...,...,...,...,...,...
305061,4908949,corner,cross_right,NaN,25150,NaN,8472,1988799
305062,4909135,corner,cross_right,NaN,118929,NaN,8472,1988799
305063,4910587,corner,cross_left,NaN,239807,NaN,8659,1988799
305064,4910843,corner,cross_right,NaN,24159,NaN,8472,1988799


In [168]:
matches.drop(['goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession', 'country_name'], axis=1, inplace=True)

In [169]:
actions['match_id']

0          489042
1          489042
2          489043
3          489044
4          489045
           ...   
305061    1988799
305062    1988799
305063    1988799
305064    1988799
305065    1988799
Name: match_id, Length: 305066, dtype: int64

In [170]:
actions['type'].unique()

array(['goal', 'card', 'foulcommit', 'shoton', 'shotoff', 'cross',
       'corner', 'throwin'], dtype=object)

In [171]:
data = matches.merge(actions, on='match_id')

In [172]:
data

,match_id,season,home_team,home_team_goal,away_team,away_team_goal,home_team_api_id,away_team_api_id,id,type,subtype1,subtype2,player1,player2,team
0,489042,2008/2009,Manchester United,1,Newcastle United,1,10260,10261,378998,goal,header,n,37799,38807,10261
1,489042,2008/2009,Manchester United,1,Newcastle United,1,10260,10261,379019,goal,shot,n,24148,24154,10260
2,489042,2008/2009,Manchester United,1,Newcastle United,1,10260,10261,379481,card,serious_fouls,y,24157,NaN,10260
3,489042,2008/2009,Manchester United,1,Newcastle United,1,10260,10261,379503,card,serious_fouls,y,30362,NaN,10260
4,489042,2008/2009,Manchester United,1,Newcastle United,1,10260,10261,379547,card,NaN,y,30829,NaN,10260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305061,1988799,2015/2016,West Bromwich Albion,1,Sunderland,0,8659,8472,4908949,corner,cross_right,NaN,25150,NaN,8472
305062,1988799,2015/2016,West Bromwich Albion,1,Sunderland,0,8659,8472,4909135,corner,cross_right,NaN,118929,NaN,8472
305063,1988799,2015/2016,West Bromwich Albion,1,Sunderland,0,8659,8472,4910587,corner,cross_left,NaN,239807,NaN,8659
305064,1988799,2015/2016,West Bromwich Albion,1,Sunderland,0,8659,8472,4910843,corner,cross_right,NaN,24159,NaN,8472


In [173]:
data.drop(['subtype1', 'subtype2', 'player1', 'player2'], axis=1, inplace=True)

In [174]:
data.dropna(inplace=True)

In [175]:
data['team'] = data['team'].astype('int64')

In [176]:
events = pd.DataFrame(0, index=data['match_id'].unique(), columns=[
      'home_team_corners', 'away_team_corners', 'home_team_crosses', 'away_team_crosses', # side plays
      'home_team_shotson', 'away_team_shotson', 'home_team_shotoff', 'away_team_shotoff', # shots on goal
      'home_team_cards', 'away_team_cards', 'home_team_fouls', 'away_team_fouls' # fouls and cards
], dtype=int).reset_index(drop=True)

In [177]:
events.insert(0, "match_id", matches['match_id'].unique(), True)
events.insert(1, "home_team_api_id", matches['home_team_api_id'], True)
events.insert(2, "away_team_api_id", matches['away_team_api_id'], True)
events

,match_id,home_team_api_id,away_team_api_id,home_team_corners,away_team_corners,home_team_crosses,away_team_crosses,home_team_shotson,away_team_shotson,home_team_shotoff,away_team_shotoff,home_team_cards,away_team_cards,home_team_fouls,away_team_fouls
0,489042,10260,10261,0,0,0,0,0,0,0,0,0,0,0,0
1,489043,9825,8659,0,0,0,0,0,0,0,0,0,0,0,0
2,489044,8472,8650,0,0,0,0,0,0,0,0,0,0,0,0
3,489045,8654,8528,0,0,0,0,0,0,0,0,0,0,0,0
4,489046,10252,8456,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,1988795,8466,8197,0,0,0,0,0,0,0,0,0,0,0,0
3036,1988796,10003,10194,0,0,0,0,0,0,0,0,0,0,0,0
3037,1988797,8586,8650,0,0,0,0,0,0,0,0,0,0,0,0
3038,1988798,9817,9825,0,0,0,0,0,0,0,0,0,0,0,0


In [178]:
data = data[data['match_id'].isin(events['match_id'])]

In [179]:
def update_events(row):
    home_team = row['team'] == row['home_team_api_id']
    event = row['type']
    match = row['match_id']

    if home_team:    
        if event=='corner':
            events.loc[events['match_id'] == match, 'home_team_corners'] += 1
        elif event=='cross':
            events.loc[events['match_id'] == match, 'home_team_crosses'] += 1
        elif event=='card':
            events.loc[events['match_id'] == match, 'home_team_cards'] += 1
        elif event=='foulcommit':
            events.loc[events['match_id'] == match, 'home_team_fouls'] += 1
        elif event=='shoton':
            events.loc[events['match_id'] == match, 'home_team_shotson'] += 1
        elif event=='shotoff':
            events.loc[events['match_id'] == match, 'home_team_shotoff'] += 1
    else:
        if event=='corner':
            events.loc[events['match_id'] == match, 'away_team_corners'] += 1
        elif event=='cross':
            events.loc[events['match_id'] == match, 'away_team_crosses'] += 1
        elif event=='card':
            events.loc[events['match_id'] == match, 'away_team_cards'] += 1
        elif event=='foulcommit':
            events.loc[events['match_id'] == match, 'away_team_fouls'] += 1
        elif event=='shoton':
            events.loc[events['match_id'] == match, 'away_team_shotson'] += 1
        elif event=='shotoff':
            events.loc[events['match_id'] == match, 'away_team_shotoff'] += 1

    return row

# Apply the function to each row of the data DataFrame

In [180]:
data.apply(update_events, axis=1)

,match_id,season,home_team,home_team_goal,away_team,away_team_goal,home_team_api_id,away_team_api_id,id,type,team
0,489042,2008/2009,Manchester United,1,Newcastle United,1,10260,10261,378998,goal,10261
1,489042,2008/2009,Manchester United,1,Newcastle United,1,10260,10261,379019,goal,10260
2,489042,2008/2009,Manchester United,1,Newcastle United,1,10260,10261,379481,card,10260
3,489042,2008/2009,Manchester United,1,Newcastle United,1,10260,10261,379503,card,10260
4,489042,2008/2009,Manchester United,1,Newcastle United,1,10260,10261,379547,card,10260
...,...,...,...,...,...,...,...,...,...,...,...
305061,1988799,2015/2016,West Bromwich Albion,1,Sunderland,0,8659,8472,4908949,corner,8472
305062,1988799,2015/2016,West Bromwich Albion,1,Sunderland,0,8659,8472,4909135,corner,8472
305063,1988799,2015/2016,West Bromwich Albion,1,Sunderland,0,8659,8472,4910587,corner,8659
305064,1988799,2015/2016,West Bromwich Albion,1,Sunderland,0,8659,8472,4910843,corner,8472


In [181]:
data.drop(['id', 'type', 'team', 'home_team_api_id', 'away_team_api_id'], axis=1, inplace=True)

In [182]:
data.drop_duplicates(subset='match_id', keep='first', inplace=True)

In [183]:
events.drop(['home_team_api_id', 'away_team_api_id'], axis=1, inplace=True)

In [185]:
premier_league_data = data.merge(events, on='match_id')

In [186]:
premier_league_data

,match_id,season,home_team,home_team_goal,away_team,away_team_goal,home_team_corners,away_team_corners,home_team_crosses,away_team_crosses,home_team_shotson,away_team_shotson,home_team_shotoff,away_team_shotoff,home_team_cards,away_team_cards,home_team_fouls,away_team_fouls
0,489042,2008/2009,Manchester United,1,Newcastle United,1,11,11,19,4,11,1,10,9,3,0,16,11
1,489043,2008/2009,Arsenal,1,West Bromwich Albion,0,12,10,16,2,12,2,13,3,0,0,11,9
2,489044,2008/2009,Sunderland,0,Liverpool,1,2,16,14,11,4,11,3,5,0,2,13,12
3,489045,2008/2009,West Ham United,2,Wigan Athletic,1,12,19,7,11,5,7,7,15,2,1,14,13
4,489046,2008/2009,Aston Villa,4,Manchester City,2,14,16,9,8,5,9,4,5,0,1,11,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,1988795,2015/2016,Southampton,2,Leicester City,2,6,7,10,21,3,9,6,10,2,0,16,10
3036,1988796,2015/2016,Swansea City,0,Stoke City,1,1,2,19,6,4,2,9,6,2,2,6,14
3037,1988797,2015/2016,Tottenham Hotspur,0,Liverpool,0,8,6,12,10,8,8,5,2,1,2,15,11
3038,1988798,2015/2016,Watford,0,Arsenal,3,3,9,13,13,4,11,5,4,2,1,11,12


In [188]:
possession.drop(['id', 'subtype1', 'team'], axis=1, inplace=True)
possession

,match_id,awaypos,homepos
0,489042,45,55
1,489043,34,66
2,489044,54,46
3,489045,48,52
4,489046,48,52
...,...,...,...
3030,1989075,39,61
3031,1989076,67,33
3032,1989077,69,31
3033,1989078,30,70


In [189]:
premier_league_data = premier_league_data.merge(possession, on='match_id')

In [190]:
premier_league_data

,match_id,season,home_team,home_team_goal,away_team,away_team_goal,home_team_corners,away_team_corners,home_team_crosses,away_team_crosses,home_team_shotson,away_team_shotson,home_team_shotoff,away_team_shotoff,home_team_cards,away_team_cards,home_team_fouls,away_team_fouls,awaypos,homepos
0,489042,2008/2009,Manchester United,1,Newcastle United,1,11,11,19,4,11,1,10,9,3,0,16,11,45,55
1,489043,2008/2009,Arsenal,1,West Bromwich Albion,0,12,10,16,2,12,2,13,3,0,0,11,9,34,66
2,489044,2008/2009,Sunderland,0,Liverpool,1,2,16,14,11,4,11,3,5,0,2,13,12,54,46
3,489045,2008/2009,West Ham United,2,Wigan Athletic,1,12,19,7,11,5,7,7,15,2,1,14,13,48,52
4,489046,2008/2009,Aston Villa,4,Manchester City,2,14,16,9,8,5,9,4,5,0,1,11,13,48,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3030,1988795,2015/2016,Southampton,2,Leicester City,2,6,7,10,21,3,9,6,10,2,0,16,10,49,51
3031,1988796,2015/2016,Swansea City,0,Stoke City,1,1,2,19,6,4,2,9,6,2,2,6,14,46,54
3032,1988797,2015/2016,Tottenham Hotspur,0,Liverpool,0,8,6,12,10,8,8,5,2,1,2,15,11,51,49
3033,1988798,2015/2016,Watford,0,Arsenal,3,3,9,13,13,4,11,5,4,2,1,11,12,67,33


In [192]:
premier_league_data.rename(columns={
    'awaypos': 'away_team_possession',
    'homepos': 'home_team_possession'
}, inplace=True)

In [193]:
premier_league_data

,match_id,season,home_team,home_team_goal,away_team,away_team_goal,home_team_corners,away_team_corners,home_team_crosses,away_team_crosses,home_team_shotson,away_team_shotson,home_team_shotoff,away_team_shotoff,home_team_cards,away_team_cards,home_team_fouls,away_team_fouls,away_team_possession,home_team_possession
0,489042,2008/2009,Manchester United,1,Newcastle United,1,11,11,19,4,11,1,10,9,3,0,16,11,45,55
1,489043,2008/2009,Arsenal,1,West Bromwich Albion,0,12,10,16,2,12,2,13,3,0,0,11,9,34,66
2,489044,2008/2009,Sunderland,0,Liverpool,1,2,16,14,11,4,11,3,5,0,2,13,12,54,46
3,489045,2008/2009,West Ham United,2,Wigan Athletic,1,12,19,7,11,5,7,7,15,2,1,14,13,48,52
4,489046,2008/2009,Aston Villa,4,Manchester City,2,14,16,9,8,5,9,4,5,0,1,11,13,48,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3030,1988795,2015/2016,Southampton,2,Leicester City,2,6,7,10,21,3,9,6,10,2,0,16,10,49,51
3031,1988796,2015/2016,Swansea City,0,Stoke City,1,1,2,19,6,4,2,9,6,2,2,6,14,46,54
3032,1988797,2015/2016,Tottenham Hotspur,0,Liverpool,0,8,6,12,10,8,8,5,2,1,2,15,11,51,49
3033,1988798,2015/2016,Watford,0,Arsenal,3,3,9,13,13,4,11,5,4,2,1,11,12,67,33


In [194]:
premier_league_data.to_csv("pl_data.csv", header=True, index=False)

In [195]:
premier_league_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3035 entries, 0 to 3034
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   match_id              3035 non-null   int64 
 1   season                3035 non-null   object
 2   home_team             3035 non-null   object
 3   home_team_goal        3035 non-null   int64 
 4   away_team             3035 non-null   object
 5   away_team_goal        3035 non-null   int64 
 6   home_team_corners     3035 non-null   int32 
 7   away_team_corners     3035 non-null   int32 
 8   home_team_crosses     3035 non-null   int32 
 9   away_team_crosses     3035 non-null   int32 
 10  home_team_shotson     3035 non-null   int32 
 11  away_team_shotson     3035 non-null   int32 
 12  home_team_shotoff     3035 non-null   int32 
 13  away_team_shotoff     3035 non-null   int32 
 14  home_team_cards       3035 non-null   int32 
 15  away_team_cards       3035 non-null   

In [196]:
premier_league_data[premier_league_data['home_team'] == 'Arsenal']

,match_id,season,home_team,home_team_goal,away_team,away_team_goal,home_team_corners,away_team_corners,home_team_crosses,away_team_crosses,home_team_shotson,away_team_shotson,home_team_shotoff,away_team_shotoff,home_team_cards,away_team_cards,home_team_fouls,away_team_fouls,away_team_possession,home_team_possession
1,489043,2008/2009,Arsenal,1,West Bromwich Albion,0,12,10,16,2,12,2,13,3,0,0,11,9,34,66
11,489133,2008/2009,Arsenal,4,Tottenham Hotspur,4,18,1,13,14,9,3,11,5,1,4,9,16,38,62
29,489152,2008/2009,Arsenal,2,Manchester United,1,9,11,12,15,6,9,6,10,3,2,17,8,48,52
40,489163,2008/2009,Arsenal,0,Aston Villa,2,15,7,26,12,8,4,5,3,2,4,10,20,47,53
70,489193,2008/2009,Arsenal,1,Wigan Athletic,0,16,6,4,7,3,7,9,7,1,2,14,15,40,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,1989040,2015/2016,Arsenal,1,Crystal Palace,1,10,5,32,15,9,2,9,3,0,2,7,15,35,65
2945,1989060,2015/2016,Arsenal,1,Norwich City,0,7,3,13,17,7,6,6,6,1,0,8,7,31,69
2965,1987597,2015/2016,Arsenal,4,Aston Villa,0,5,4,16,6,7,4,8,0,0,1,10,6,40,60
2985,1988750,2015/2016,Arsenal,2,Stoke City,0,13,1,15,7,17,5,11,1,1,1,6,13,31,69


In [201]:
premier_league_data[(premier_league_data['home_team'] == 'Arsenal') & (premier_league_data['away_team'] == 'Tottenham Hotspur')]

,match_id,season,home_team,home_team_goal,away_team,away_team_goal,home_team_corners,away_team_corners,home_team_crosses,away_team_crosses,home_team_shotson,away_team_shotson,home_team_shotoff,away_team_shotoff,home_team_cards,away_team_cards,home_team_fouls,away_team_fouls,away_team_possession,home_team_possession
11,489133,2008/2009,Arsenal,4,Tottenham Hotspur,4,18,1,13,14,9,3,11,5,1,4,9,16,38,62
403,658716,2009/2010,Arsenal,3,Tottenham Hotspur,0,12,4,6,13,8,6,8,2,1,1,16,10,44,56
806,839945,2010/2011,Arsenal,2,Tottenham Hotspur,3,4,12,46,21,6,5,6,4,3,1,19,17,47,53
1315,1025556,2011/2012,Arsenal,5,Tottenham Hotspur,2,13,9,17,14,8,4,11,7,3,4,12,10,44,56
1545,1229218,2012/2013,Arsenal,5,Tottenham Hotspur,2,10,12,26,8,5,4,3,3,1,3,12,13,46,54
2115,1474151,2013/2014,Arsenal,1,Tottenham Hotspur,0,6,11,7,16,6,8,6,8,2,2,14,14,56,44
2615,1724032,2014/2015,Arsenal,1,Tottenham Hotspur,1,15,5,30,6,15,6,2,1,3,6,8,14,31,69
2685,1988820,2015/2016,Arsenal,1,Tottenham Hotspur,1,7,6,16,11,2,4,5,9,0,1,13,17,45,55
